In [34]:
# Imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

from xgboost import XGBClassifier
import warnings; warnings.filterwarnings("ignore", category=RuntimeWarning)
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'ignore', 'under': 'ignore', 'invalid': 'warn'}

## 1) Load Processed Data

In [35]:
# In[2]: Robust Data Import
FILENAME = "../../processed_online_shoppers_data.csv"

df = pd.read_csv(FILENAME)
print(f"Loaded dataset from: {FILENAME}")

print("Shape:", df.shape)
df.head()

Loaded dataset from: ../../processed_online_shoppers_data.csv
Shape: (12330, 39)


,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_Other,VisitorType_Returning_Visitor,Revenue
0,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
1,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.317178,-0.308821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
2,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
3,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.622954,0.573535,1.994610,-0.317178,-0.308821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False
4,-0.696993,-0.457191,-0.396478,-0.244931,-0.488636,-0.296430,-0.045196,0.142551,-0.317178,-0.308821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False


In [36]:
# Target and features
y = df["Revenue"].astype(int)
X = df.drop(columns=["Revenue"])

# Identify categorical vs numeric columns
cat_cols = [c for c in X.columns if X[c].dtype == "object" or X[c].dtype == "bool"]
num_cols = [c for c in X.columns if c not in cat_cols]

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

## 2. Training Decision Tree Models with GridSearch

In [37]:
# Preprocessing: scale numerics; one-hot encode categoricals
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_cols)
    ],
    remainder="drop",
    sparse_threshold=0.3
)

# Base model
dt = DecisionTreeClassifier(random_state=42, class_weight="balanced")

# Grid (like Lab 6)
param_grid_dt = {
    "clf__criterion": ["gini", "entropy"],
    "clf__max_depth": [2, 4, 6, 8, 10],
    "clf__min_samples_split": [2, 5, 10],
}

pipe_dt = Pipeline([("pre", preprocess), ("clf", dt)])

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_dt = GridSearchCV(
    estimator=pipe_dt,
    param_grid=param_grid_dt,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1
)

grid_dt.fit(X_train, y_train)
print("DT best CV score:", grid_dt.best_score_)
print("DT best params:", grid_dt.best_params_)

DT best CV score: 0.8743917274939172
DT best params: {'clf__criterion': 'gini', 'clf__max_depth': 2, 'clf__min_samples_split': 2}


In [38]:
model_tree = grid_dt.best_estimator_
y_pred_dt   = model_tree.predict(X_test)               # labels for accuracy
y_proba_dt  = model_tree.predict_proba(X_test)[:, 1]   # scores for AUC/PR-AUC

print(f"Decision Tree - Test Accuracy: {accuracy_score(y_test, y_pred_dt):.2%}")
print(f"Decision Tree - Test ROC-AUC : {roc_auc_score(y_test, y_proba_dt):.2f}")
print(f"Decision Tree - Test PR-AUC  : {average_precision_score(y_test, y_proba_dt):.2f}")

Decision Tree - Test Accuracy: 86.90%
Decision Tree - Test ROC-AUC : 0.90
Decision Tree - Test PR-AUC  : 0.55


### 3. Training XGBoost with GridSearch

In [39]:
# Class imbalance helper (pos/neg ratio for XGB)
pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = neg / max(pos, 1)

xgb = XGBClassifier(
    random_state=42,
    eval_metric="logloss",
    nthread=-1,
    tree_method="hist",
    scale_pos_weight=scale_pos_weight
)

param_grid_xgb = {
    "xgb__n_estimators": [50, 100, 200],
    "xgb__max_depth": [3, 5, 7],
    "xgb__learning_rate": [0.01, 0.1, 0.3],
}

pipe_xgb = Pipeline([("pre", preprocess), ("xgb", xgb)])

grid_xgb = GridSearchCV(
    estimator=pipe_xgb,
    param_grid=param_grid_xgb,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1
)

grid_xgb.fit(X_train, y_train)
print("XGB best CV score:", grid_xgb.best_score_)
print("XGB best params:", grid_xgb.best_params_)

XGB best CV score: 0.8946674776966748
XGB best params: {'xgb__learning_rate': 0.3, 'xgb__max_depth': 7, 'xgb__n_estimators': 200}


In [40]:
# Evaluate on test set
best_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)
y_proba_xgb = best_xgb.predict_proba(X_test)[:, 1]

print(f"XGBoost - Test Accuracy: {accuracy_score(y_test, y_pred_xgb):.2%}")
print(f"XGBoost - Test ROC-AUC : {roc_auc_score(y_test, y_proba_xgb):.2f}")

XGBoost - Test Accuracy: 88.93%
XGBoost - Test ROC-AUC : 0.91


## 4) Compare Models

In [41]:
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred_dt):.2%}, AUC: {roc_auc_score(y_test, y_proba_dt):.2f}")
print(f"XGBoost       Accuracy: {accuracy_score(y_test, y_pred_xgb):.2%}, AUC: {roc_auc_score(y_test, y_proba_xgb):.2f}")

Decision Tree Accuracy: 86.90%, AUC: 0.90
XGBoost       Accuracy: 88.93%, AUC: 0.91
